# Load Data

In [1]:
# where we will unpack data
# OUTPUT_FOLDER = "/kaggle/working/"
# DATA_FOLDER = "/kaggle/input/solfune-satellite/"
# TRAIN_DATA_FOLDER = DATA_FOLDER + 'train/'

OUTPUT_FOLDER = "/scratch/aakash_ks.iitr"
DATA_FOLDER = "/scratch/aakash_ks.iitr/data/solafune/"
TRAIN_DATA_FOLDER = DATA_FOLDER + 'train/'

# TEST_DATA_FOLDER = DATA_FOLDER + 'test/'

# Imports

In [ ]:
import os
import random
import tifffile
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

In [ ]:
import torch
import torch.nn.functional as F
import torch.nn as nn

from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
from torchvision.transforms import v2

import timm

In [21]:
import torch

class CFG:
    seed = 42
    N_folds = 5
    train_folds = [0, 1] # [0,1,2,3,4]

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    apex=True # use half precision

    # model_name = "maxvit_tiny_tf_512"
    model_name = "maxvit_tiny_tf_224.in1k"
    epochs = 5
    weights =  torch.tensor([0.206119, 0.793881],dtype=torch.float32)

    clip_val = 1000.
    batch_size = 16
    gradient_accumulation_steps = 1

    lr = 1e-4
    weight_decay=1e-2
    
    resolution = 224
    channels = 14
    comments = "fe same as baseline"

In [ ]:
import wandb
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# wandb.login(key=user_secrets.get_secret("wandb_api"))

run = wandb.init(
    project="hello-world", 
    dir=OUTPUT_FOLDER,
    config={
    k:v for k, v in CFG.__dict__.items() if not k.startswith('__')}
)

# Load train data

In [ ]:
train_data = pd.read_csv(os.path.join(DATA_FOLDER, 'answer.csv'), names=["file_name", "label"], header=None)

# Dataset

In [ ]:
def feature_engineering(image):
    endmi = ((image[..., 7] + image[..., 8]) - (image[..., 10] + image[..., 11])) / (image[..., 7] + image[..., 8] + image[..., 10] + image[..., 10] + image[..., 11] + 1e-10)
    ndvi = (image[..., 7] - image[..., 3]) / (image[..., 7] + image[..., 3] + 1e-10)
    # Land Use Visualization for Sentinel-2 Using Linear Discriminant Analysis Script
    luvlda1 = abs(0.4809 * ((((image[...,1] * 2.5) - 0.3329) * -9.4425) + (((image[...,2] * 2.5) - 0.3182) * 2.1846) + (((image[...,3] * 2.5) - 0.3380) * 2.5333) + (((image[...,10] * 2.5) - 0.5644) * 9.9256) + (((image[...,11] * 2.5) - 0.4216) * -13.6911)) + -0.4766)
    luvlda2 = abs(0.3275 * ((((image[...,1] * 2.5) - 0.2844) * 13.3644) + (((image[...,2] * 2.5) - 0.2736) * -6.6588) + (((image[...,3] * 2.5) - 0.2750) * -1.1994) + (((image[...,7] * 2.5) - 0.5972) * -0.2090) + (((image[...,8] * 2.5) - 0.6648) * 5.1630) + (((image[...,10] * 2.5) - 0.5651) * -7.2183)) + 0.0463)
    luvlda3 = abs(0.2361 * ((((image[...,2] * 2.5) - 0.2429) * 21.8759) + (((image[...,3] * 2.5) - 0.2321) * -6.0679) + (((image[...,7] * 2.5) - 0.4371) * -3.0608) + (((image[...,10] * 2.5) - 0.4146) * -4.4420)) + 0.2061)
    # Alteration and Gossan
    alt1 = image[...,10]/(image[...,11]+1e-10)
    alt2 = image[...,11]/(image[...,10]+1e-10)
    goss = image[...,10]/(image[...,3]+1e-10)
    image = np.concatenate(
        [
            image * 2 - 1, # scale band data from -1 to 1
            np.expand_dims(endmi, axis=-1),
            np.expand_dims(ndvi, axis=-1),
            # np.expand_dims(luvlda1, axis=-1),
            # np.expand_dims(luvlda2, axis=-1),
            # np.expand_dims(luvlda3, axis=-1),
            # np.expand_dims(alt1, axis=-1),
            # np.expand_dims(alt2, axis=-1),
            # np.expand_dims(goss, axis=-1),
        ],
        axis=-1,
    )
    return image

In [ ]:
min_values = np.array([-1, -1, -1, -1, 
                       -1, -1, -1, -1, 
                       -1, -1, -1, -1,
                       -1, -1,  
                #        0,  0,
                #         0,  0,  0,  0
                        ])

max_values = np.array([1, 1, 1, 1,
        1, 1, 1, 1,
        1, 1, 1, 1,
        1, 1, 
        # 20, 10,
        # 10, 10, 10, 10
        ])

In [ ]:
mean = (max_values + min_values) / 2
std = (max_values - min_values) / 2

train_transforms = v2.Compose([
    v2.ToTensor(),
    v2.Resize(CFG.resolution),
    v2.Normalize(mean, std),
    v2.RandomRotation(degrees=90),
    v2.RandomHorizontalFlip(),
    v2.RandomVerticalFlip(),
    v2.ToDtype(torch.float32, scale=False),
])

val_transforms = v2.Compose([
    v2.ToTensor(),
    v2.Resize(CFG.resolution),
    v2.Normalize(mean, std),
    v2.ToDtype(torch.float32, scale=False),
])

In [ ]:
class ImageTrainDataset(Dataset):
    def __init__(
        self,
        folder,
        data,
        transforms,
    ):
        self.folder = folder
        self.data = data
        self.transforms = transforms

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        d = self.data.loc[index]
        image = tifffile.imread(f"{self.folder}{d.file_name}")
        image = feature_engineering(np.array(image))
        image = np.clip(image, -2, 25)
        image = self.transforms(image)
        label = d.label

        return {"image": image, "label": torch.tensor(label, dtype=torch.long)}

# Metric

In [ ]:
from sklearn.metrics import f1_score as sklearn_f1
from sklearn.metrics import confusion_matrix, roc_auc_score

In [ ]:
def find_best_threshold(targets, predictions):
    score_5 = sklearn_f1(targets, predictions > 0.5)
    best_score = 0
    best_th = -1
    for i in range(100):
        threshold =  i/100
        _score = sklearn_f1(targets, predictions > threshold)
        if _score > best_score:
            best_score = _score
            best_th = threshold

    tn, fp, fn, tp = confusion_matrix(targets.numpy(), predictions.numpy() > best_th).ravel()
    print(f"tp: {tp}, tn: {tn}, fp: {fp}, fn: {fn}")
    return score_5, best_score, best_th

# Train and evaluate functions

In [ ]:
class style:
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    END = '\033[0m'
    BOLD = '\033[1m'

In [ ]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

In [ ]:
def evaluate_model(cfg, model, data_loader, epoch=-1):
    loss_fn = nn.CrossEntropyLoss(weight=cfg.weights.to(cfg.device), label_smoothing=0.1)

    model.eval()
    val_loss = 0

    targets = []
    predictions = []

    total_len = len(data_loader)
    tk0 = tqdm(enumerate(data_loader), total=total_len)
    for step, data in tk0:
        input = data["image"].to(cfg.device, non_blocking=True)
        target = data["label"].to(cfg.device, non_blocking=True)

        with torch.no_grad():
            logits = model(input)

        loss = loss_fn(logits, target)
        val_loss += loss.item()

        targets.append(target.detach().cpu())
        predictions.append(logits.detach().cpu())
        del input, target, logits

    targets = torch.cat(targets, dim=0)
    predictions = torch.cat(predictions, dim=0)
    predictions = F.sigmoid(predictions)

    val_loss /= total_len
    base_score, best_score, best_th = find_best_threshold(targets, predictions[:, 1])
    roc_auc = roc_auc_score(targets.numpy(), predictions[:, 1].numpy())

    print(f'Epoch {epoch} validation loss = {val_loss:.4f}, base f1 score (0.5 threshold) = {base_score:.4f} (best threshold: {best_th} -> f1 {best_score:.4f}), auc = {roc_auc:.4f}')
    return val_loss, best_score, roc_auc

In [ ]:
def train_epoch(cfg, model, train_loader, optimizer, scheduler, epoch):
    scaler = torch.cuda.amp.GradScaler(enabled=cfg.apex)
    loss_fn = nn.CrossEntropyLoss(weight=cfg.weights.to(cfg.device), label_smoothing=0.1)

    model.train()
    train_loss = 0
    learning_rate_history = []

    targets = []
    predictions = []

    total_len = len(train_loader)
    tk0 = tqdm(enumerate(train_loader), total=total_len)
    for step, data in tk0:
        input = data["image"].to(cfg.device, non_blocking=True)
        target = data["label"].to(cfg.device, non_blocking=True)

        # https://pytorch.org/blog/accelerating-training-on-nvidia-gpus-with-pytorch-automatic-mixed-precision/
        with torch.cuda.amp.autocast(enabled=cfg.apex):
            logits = model(input)
            loss = loss_fn(logits, target)

        scaler.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=cfg.clip_val)

        train_loss += loss.item()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

        if scheduler is None:
            lr = optimizer.param_groups[0]['lr']
        else:
            scheduler.step()
            lr = scheduler.get_last_lr()[0]

        tk0.set_description(f"Epoch {epoch} training {step+1}/{total_len} [LR {lr:0.6f}] - loss: {train_loss/(step+1):.4f}")
        learning_rate_history.append(lr)

        targets.append(target.detach().cpu())
        predictions.append(logits.detach().cpu())
        del input, target

    targets = torch.cat(targets, dim=0)
    predictions = torch.cat(predictions, dim=0)
    predictions = F.sigmoid(predictions)

    train_loss /= total_len
    base_score, best_score, best_th = find_best_threshold(targets, predictions[:, 1])
    roc_auc = roc_auc_score(targets.numpy(), predictions[:, 1].numpy())

    print(f'Epoch {epoch} train loss = {train_loss:.4f}, base f1 score (0.5 threshold) = {base_score:.4f} (best threshold: {best_th} -> f1 {best_score:.4f}), auc = {roc_auc:.4f}')
    return train_loss, best_score, learning_rate_history, roc_auc

# Train model

## Split data

The distribution of classes in the training data is not balance so using StratifiedKFold will ensure that the distrubution of positive and negative samples in all folds will match the original distributions.

In [ ]:
# plt.figure(figsize=(4,2))
# sns.histplot(train_data["label"])

In [ ]:
from sklearn.model_selection import StratifiedKFold

sgkf = StratifiedKFold(n_splits=CFG.N_folds, random_state=CFG.seed, shuffle=True)
for i, (train_index, test_index) in enumerate(sgkf.split(train_data["file_name"].values, train_data["label"].values)):
    train_data.loc[test_index, "fold"] = i

In [ ]:
# from torchgeo import models
# from torch import nn

In [ ]:
def create_model():
    model = timm.create_model(CFG.model_name, in_chans=CFG.channels, num_classes=2, pretrained=True)

#     model = models.resnet50(models.ResNet50_Weights.SENTINEL2_ALL_DINO)
#     wd = torch.concat([model.conv1.weight[:, :13, ...], model.conv1.weight[:, :7, ...]], dim=1)
#     model.conv1 = nn.Conv2d(20, 64, 7, 2, 3, bias=False)
#     model.conv1.weight = nn.Parameter(wd)
#     model.fc = nn.Linear(in_features=2048, out_features=2, bias=True)

    return model.to(CFG.device)

## Train folds

In [ ]:
for FOLD in CFG.train_folds:

    seed_everything(CFG.seed)

    # PREPARE DATA
    fold_train_data = train_data[train_data["fold"] != FOLD].reset_index(drop=True)
    fold_valid_data = train_data[train_data["fold"] == FOLD].reset_index(drop=True)

    # display(
    #     pd.merge(
    #         fold_valid_data.groupby(by=["label"])["file_name"].count().rename("valid").reset_index(),
    #         fold_train_data.groupby(by=["label"])["file_name"].count().rename("train").reset_index(),
    #          on="label", how="left").T,)


    train_dataset = ImageTrainDataset(TRAIN_DATA_FOLDER, fold_train_data, transforms=train_transforms)
    valid_dataset = ImageTrainDataset(TRAIN_DATA_FOLDER, fold_valid_data, transforms=val_transforms)

    train_loader = DataLoader(
            train_dataset,
            batch_size=CFG.batch_size,
            shuffle=True,
            num_workers=os.cpu_count(),
            pin_memory=True,
            drop_last=True
        )

    valid_loader = DataLoader(
        valid_dataset,
        batch_size=CFG.batch_size,
        shuffle=False,
        num_workers=os.cpu_count(),
        pin_memory=True,
        drop_last=False,
    )

    # PREPARE MODEL, OPTIMIZER AND SCHEDULER
    model = create_model()
    print(f"Model parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):_}")

    optimizer = torch.optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, eta_min=1e-6, T_max =CFG.epochs * len(train_loader),
        )

    # TRAIN FOLD
    learning_rate_history = []
    train_loss_history = []
    train_score_history = []
    val_loss_history = []
    val_score_history = []

    best_score = 0
    
    wandb.run.tags = [f"fold_{FOLD}"]
    
    for epoch in range(0, CFG.epochs):
        train_loss, train_score, train_lr, train_auc = train_epoch(CFG, model, train_loader, optimizer, scheduler, epoch)
        train_loss_history.append(train_loss)
        train_score_history.append(train_score)
        learning_rate_history.extend(train_lr)

        val_loss, val_score, val_auc = evaluate_model(CFG, model, valid_loader, epoch)
        val_loss_history.append(val_loss)
        val_score_history.append(val_score)
        
        wandb.log(
            {'train': {'loss': train_loss, 'f1': train_score, 'auc': train_auc}, 
             'val': {'loss': val_loss, 'f1': val_score, 'auc': val_auc}})

        if (val_score > best_score):
            print(f"{style.GREEN}New best score: {best_score:.4f} -> {val_score:.4f}{style.END}")
            best_score = val_score
            torch.save(model.state_dict(), os.path.join(wandb.run.dir, f'best_model_fold_{FOLD}.pth'))
            
    # run.log_model(
    #     path=os.path.join(wandb.run.dir, 'best_model_fold_{FOLD}'), 
    #     name=f'{CFG.model_name}_fold_{FOLD}'
    # )

    # # plot train and validation loss, score and LR
    # fig, axes = plt.subplots(1,3, figsize=(12,3))
    # axes[0].plot(train_loss_history, label="Train")
    # axes[0].plot(val_loss_history, label="Valid")
    # axes[0].title.set_text("Loss")
    # axes[0].set_xlabel("Epoch")
    # axes[0].legend()

    # axes[1].plot(train_score_history, label="Train")
    # axes[1].plot(val_score_history, label="Valid")
    # axes[1].title.set_text("F1 score")
    # axes[1].set_xlabel("Epoch")
    # axes[1].legend()

    # axes[2].plot(learning_rate_history, label="LR")
    # axes[2].legend()
    # axes[2].title.set_text("Learning rate")
    # axes[2].set_xlabel("Step")
    # fig.suptitle(f"Fold {FOLD}")
    # fig.tight_layout()
    # plt.show()

In [ ]:
wandb.finish()